In [7]:
from google.colab import files

uploaded = files.upload()

Saving words_250000_train.txt to words_250000_train (1).txt


In [8]:
#Importing all the necessary libraries
import math
import requests
import json
import random
import collections
import string
import time
import matplotlib.pyplot as plt
import re
import secrets
import re
from collections import Counter, defaultdict, OrderedDict

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [9]:
observation={}

for i in range(1,50):
    observation[i] = 0

observation_correct = {}
for i in range(1,50):
    observation_correct[i] = 0

In [10]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        self.word_len = None

        self.min_word_length = None
        self.max_word_length = None

        file_path = 'words_250000_train.txt'

        with open(file_path, 'r') as file:
            file_contents = file.read()

        words = file_contents.split()
        word_freq = Counter(words)

        n_letter_word_freq = defaultdict(int)

        for word, freq in word_freq.items():
            n = len(word)
            n_letter_word_freq[n] += freq

        self.min_word_length = min(n_letter_word_freq.keys())
        self.max_word_length = max(n_letter_word_freq.keys())

        self.letter_range = 'abcdefghijklmnopqrstuvwxyz'

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
        self.ngram_freq_list = []
        self.prefix_freq_list = []
        self.suffix_freq_list = []

        def create_freq(dictionary, n):

            ngram_freq = {}
            prefix_freq = {}
            suffix_freq = {}

            for words in dictionary:
                if (len(words) < n):
                    continue

                for i in range(len(words)-n+1):
                    if (words[i:i+n] not in ngram_freq.keys()):
                        ngram_freq[words[i:i+n]] = 0
                    ngram_freq[words[i:i+n]] += 1

                    if (words[0:0+n] not in prefix_freq.keys()):
                        prefix_freq[words[0:0+n]] = 0
                    prefix_freq[words[0:0+n]] += 1

                    if (words[-n:] not in suffix_freq.keys()):
                        suffix_freq[words[-n:]] = 0
                    suffix_freq[words[-n:]] += 1


            return ngram_freq, prefix_freq, suffix_freq


        for i in range(6):
            temp1, temp2, temp3 = create_freq(self.full_dictionary, i+1)
            self.ngram_freq_list.append(temp1)
            self.prefix_freq_list.append(temp2)
            self.suffix_freq_list.append(temp3)

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")

        # find length of passed word
        len_word = len(clean_word)

        #  get word_len
        self.word_len = len(clean_word)

        # storing list of letters which are not present in the final answer
        letters_not_found = set()
        for letter_guess in self.guessed_letters:
            if letter_guess not in clean_word:
                letters_not_found.add(letter_guess)

        blank_frac = clean_word.count('.')/len_word

        #calculate vowel and consonant count
        vowels = "aeiou"
        vowel_count = 0
        for l in clean_word:
            if l in vowels:
                vowel_count = vowel_count + 1

        consonant_count = len(clean_word) - vowel_count

        if consonant_count != 0:
            vowel_consonant_ratio = vowel_count / consonant_count
        else:
            vowel_consonant_ratio = float('inf')

        life_len = 6 - len(letters_not_found)

        current_dictionary = self.current_dictionary
        new_dictionary = []
        for dict_word in current_dictionary:
            if len(dict_word) != len_word:
                continue
            else:
                if re.match(clean_word,dict_word):
                    add = True
                    temp = list(letters_not_found)
                    for char in temp:
                        if char in dict_word:
                            add = False
                            break

                    if (add == True):
                        new_dictionary.append(dict_word)

        self.current_dictionary = new_dictionary

        threshold_blank = {}
        min_wordlen = self.min_word_length
        max_wordlen = self.max_word_length

        for i in range(1,5):
            threshold_blank[i] = 1
        for i in range(5,9):
            threshold_blank[i] = 0.85
        for i in range(9,50):
            threshold_blank[i] = 1

        threshold_life = {}
        min_wordlen = self.min_word_length
        max_wordlen = self.max_word_length

        for i in range(1,7):
            threshold_life[i] = 4
        for i in range(7,12):
            threshold_life[i] = 5
        for i in range(12,50):
            threshold_life[i] = 5



        if (blank_frac > threshold_blank[len_word] and len(new_dictionary) != 0):

            letter_range = self.letter_range
            InfoGain_letterwise = {}

            for l in letter_range:
                n = 0

                for word in new_dictionary:
                    letter_occurrence = word.count(l)
                    if letter_occurrence > n:
                        n = letter_occurrence

                subset_store = {}
                subset_store['x'] = len(new_dictionary)
                notcount = len(new_dictionary)
                for i in range(1, n+1):
                    subset_store['x_i'] = 0

                for word in new_dictionary:
                    cnt = word.count(l)
                    key = 'x_' + str(cnt)
                    if key not in subset_store:
                        subset_store[key] = 0

                    subset_store[key] += 1
                    if cnt > 0:
                        notcount = notcount - 1

                subset_store['x_not_l'] = notcount
                subset_store['x_l'] = len(new_dictionary) - notcount


                # now, we have a dictionary with counts of x_l, x_not_l, x_i to x_n
                # Next step is to calculate the entropy

                entropy_l = 0
                for i in range(1, n+1):
                    if subset_store['x_i'] > 0:
                        cause = ((subset_store['x_i'] / len(new_dictionary))*(i / len_word)*(i / len_word))
                        var1 = math.log(cause)
                        var = math.log(2)
                        effect = -1 * (cause * (var1 / var))
                        entropy_l = entropy_l + effect

                # now, let's calculate the information gain
                # we have to set a penalising factor (ideally less than 1 to reduce it's effect on the gain)
                # since wrong guess must be avoided

                if life_len > 5:
                    penalizing_factor = 0.1
                else:
                    penalizing_factor = 0.01

                if subset_store['x_not_l'] > 0:
                    InfoGain = (((subset_store['x_l'] / len(new_dictionary)) * entropy_l) + (penalizing_factor) * (subset_store['x_not_l'] / len(new_dictionary)) * (-1) * (subset_store['x_not_l'] / len(new_dictionary)) * math.log2 (subset_store['x_not_l'] / len(new_dictionary)))
                else:
                    InfoGain = 1
                # input the IG in the dictionary

                InfoGain_letterwise[l] = InfoGain

            #now, in order to find the maximum of Information gain and to return that letter
            # We are going to sort the dictionary in descending order
            sorted_items = sorted(InfoGain_letterwise.items(), key=lambda item: item[1], reverse=True)
            sorted_dict = OrderedDict(sorted_items)

            for key in sorted_dict:
                if key not in self.guessed_letters:
                    return key

        else:
            freq_table_lat = {}
            for alphabet in list(string.ascii_lowercase):
                freq_table_lat[alphabet] = 0

            weights_of_ngrams = [0.1, 1, 10, 100, 1000, 10000]
            prefix_weights = [0.1, 1, 10, 100, 1000, 10000]
            suffix_weights = [0.1, 1, 10, 100, 1000, 10000]


        if (len_word > 6):
            for i in range(6):
                temp = self.ngram_freq_list[i]
                for ngram, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(ngram)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=2):
                        if (re.search(curr, temp_string)):
                            itr = re.finditer(curr, temp_string)
                            for match in itr:
                                for guess in can_be_guessed_letters:
                                    freq_table_lat[guess] += (weights_of_ngrams[i]*freq)
                            break

                temp = self.prefix_freq_list[i]
                for pref, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(pref)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=2):
                        if (re.match(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (prefix_weights[i]*freq)
                            break

                temp = self.suffix_freq_list[i]
                for suff, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(suff)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    curr = curr+'$'
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=2):
                        if (re.search(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (suffix_weights[i]*freq)
                            break


            median_vc = {} #Represents a threshold of vowel:consonant ratio

            for i in range(1,29):
                median_vc[i] = 1.5;

            if vowel_consonant_ratio > median_vc[len_word] and life_len < 4:
                for vowel in vowels:
                    if vowel not in self.guessed_letters:
                        freq_table_lat[vowel] *= 0.1


            for letter, _ in sorted(freq_table_lat.items(), key=lambda x: (x[1], x[0]), reverse=True):
                if letter not in self.guessed_letters:
                    return letter

        if (len_word < 5):
            for i in range(6):
                temp = self.ngram_freq_list[i]
                for ngram, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(ngram)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0):
                        if (re.search(curr, temp_string)):
                            itr = re.finditer(curr, temp_string)
                            for match in itr:
                                for guess in can_be_guessed_letters:
                                    freq_table_lat[guess] += (weights_of_ngrams[i]*freq)
                            break

                temp = self.prefix_freq_list[i]
                for pref, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(pref)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0):
                        if (re.match(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (prefix_weights[i]*freq)
                            break

                temp = self.suffix_freq_list[i]
                for suff, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(suff)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    curr = curr+'$'
                    if (skip == False and len(can_be_guessed_letters)!=0):
                        if (re.search(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (suffix_weights[i]*freq)
                            break


            median_vc = {}

            for i in range(1,29):
                median_vc[i] = 1.5;

            for i in range(3,10):
                median_vc[3] = 0.5;
                median_vc[4] = 0.33;

            if vowel_consonant_ratio > median_vc[len_word] and life_len < 5:
                for vowel in vowels:
                    if vowel not in self.guessed_letters:
                        freq_table_lat[vowel] *= 0.1


            for letter, _ in sorted(freq_table_lat.items(), key=lambda x: (x[1], x[0]), reverse=True):
                if letter not in self.guessed_letters:
                    return letter

        else:
            for i in range(6):
                temp = self.ngram_freq_list[i]
                for ngram, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(ngram)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=3):
                        if (re.search(curr, temp_string)):
                            #print(ngram)
                            itr = re.finditer(curr, temp_string)
                            for match in itr:
                                for guess in can_be_guessed_letters:
                                    freq_table_lat[guess] += (weights_of_ngrams[i]*freq)
                            break

                temp = self.prefix_freq_list[i]
                for pref, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(pref)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=3):
                        if (re.match(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (prefix_weights[i]*freq)
                            break

                temp = self.suffix_freq_list[i]
                for suff, freq in sorted(temp.items(), key=lambda x: (x[1], x[0]), reverse=True):
                    curr = list(suff)
                    can_be_guessed_letters = []
                    skip = False
                    for position in range(i+1):
                        if curr[position] in letters_not_found:
                            skip = True
                            break
                        elif curr[position] not in self.guessed_letters:
                            can_be_guessed_letters.append(curr[position])
                            curr[position] = '!'

                    temp_string = clean_word.replace('.', '!')
                    curr = "".join(curr)
                    curr = curr+'$'
                    if (skip == False and len(can_be_guessed_letters)!=0 and len(can_be_guessed_letters)<=3):
                        if (re.search(curr, temp_string)):
                            for guess in can_be_guessed_letters:
                                freq_table_lat[guess] += (suffix_weights[i]*freq)
                            break


            median_vc = {}

            for i in range(1,29):
                median_vc[i] = 1.5;

            for i in range(3,10):
                median_vc[3] = 0.5;
                median_vc[4] = 0.33;
                median_vc[5] = 0.67;
                median_vc[6] = 0.5

            if vowel_consonant_ratio > median_vc[len_word] and life_len < 5:
                for vowel in vowels:
                    if vowel not in self.guessed_letters:
                        freq_table_lat[vowel] *= 0.1


            for letter, _ in sorted(freq_table_lat.items(), key=lambda x: (x[1], x[0]), reverse=True):
                if letter not in self.guessed_letters:
                    return letter


        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                    return letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    observation[self.word_len] += 1
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))

                    observation[self.word_len] += 1
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [11]:
api = HangmanAPI(access_token="1520c56f4c2b55bd0c07b624590e9f", timeout=2000)

In [12]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game


KeyboardInterrupt: 

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)